In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

import numpy as np
from PIL import Image

from sklearn.metrics import f1_score, precision_score, recall_score

In [2]:
class Config:
    img_size_global = 224
    img_size_local = 96
    num_patches = 4
    patch_size = 96
    batch_size = 8
    use_smart_patches = True

In [3]:
class SmartPatchExtractor:
    def __init__(self, patch_size=96, num_patches=4):
        self.patch_size = patch_size
        self.num_patches = num_patches

    def extract_patches(self, pil_img):
        w, h = pil_img.size
        ps = self.patch_size
        patches = []
        for _ in range(self.num_patches):
            x = np.random.randint(0, max(1, w - ps))
            y = np.random.randint(0, max(1, h - ps))
            patch = pil_img.crop((x, y, x + ps, y + ps))
            patches.append(patch)
        return patches

In [4]:
def get_transforms():
    train_transform = transforms.Compose([
        transforms.Resize((Config.img_size_global, Config.img_size_global)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225]),
    ])

    val_transform = transforms.Compose([
        transforms.Resize((Config.img_size_global, Config.img_size_global)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]),
    ])

    local_transform = transforms.Compose([
        transforms.Resize((Config.img_size_local, Config.img_size_local)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]),
    ])

    return train_transform, val_transform, local_transform

transform_global_train, transform_global_val, transform_local = get_transforms()

In [5]:
class SmartLeafDataset(Dataset):
    def __init__(self, root_dir, transform_global, transform_local,
                 num_patches=4, patch_size=96, use_smart_patches=True):
        self.root_dir = root_dir
        self.transform_global = transform_global
        self.transform_local = transform_local
        self.num_patches = num_patches
        self.patch_size = patch_size
        self.use_smart_patches = use_smart_patches

        if use_smart_patches:
            self.patch_extractor = SmartPatchExtractor(patch_size, num_patches)

        self.samples = []
        self.classes = sorted([d for d in os.listdir(root_dir)
                              if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for cls in self.classes:
            cls_dir = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.samples.append((os.path.join(cls_dir, fname), 
                                         self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def _extract_random_patches(self, pil_img):
        w, h = pil_img.size
        ps = self.patch_size
        patches = []
        for _ in range(self.num_patches):
            x = np.random.randint(0, max(1, w - ps))
            y = np.random.randint(0, max(1, h - ps))
            patch = pil_img.crop((x, y, x + ps, y + ps))
            patches.append(patch)
        return patches

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('RGB')

        img_global = self.transform_global(img)

        if self.use_smart_patches:
            try:
                raw_patches = self.patch_extractor.extract_patches(img)
            except:
                raw_patches = self._extract_random_patches(img)
        else:
            raw_patches = self._extract_random_patches(img)

        local_tensors = [self.transform_local(p) for p in raw_patches]
        patches_tensor = torch.stack(local_tensors, dim=0)

        return img_global, patches_tensor, label

In [6]:
TRAIN_DIR = r"C:\Users\ENNHILI YASSINE\Desktop\ASMAE-ABDELOUAFI\train"
VAL_DIR = r"C:\Users\ENNHILI YASSINE\Desktop\ASMAE-ABDELOUAFI\val"
TEST_DIR = r"C:\Users\ENNHILI YASSINE\Desktop\ASMAE-ABDELOUAFI\test"


def make_dataloaders():
    train_ds = SmartLeafDataset(
        TRAIN_DIR, transform_global_train, transform_local,
        num_patches=Config.num_patches, patch_size=Config.img_size_local
    )
    val_ds = SmartLeafDataset(
        VAL_DIR, transform_global_val, transform_local,
        num_patches=Config.num_patches, patch_size=Config.img_size_local
    )
    test_ds = SmartLeafDataset(
        TEST_DIR, transform_global_val, transform_local,
        num_patches=Config.num_patches, patch_size=Config.img_size_local
    )

    train_loader = DataLoader(train_ds, batch_size=Config.batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=Config.batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=Config.batch_size, shuffle=False)

    print("Classes :", len(train_ds.classes))
    return train_loader, val_loader, test_loader, train_ds.classes

train_loader, val_loader, test_loader, class_names = make_dataloaders()
num_classes = len(class_names)

Classes : 38


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

ckpt_path = r"C:\Users\ENNHILI YASSINE\Desktop\ASMAE-ABDELOUAFI\CV_Project\efficientnet_b0_best.pth"
state = torch.load(ckpt_path, map_location=device)

if "model_state_dict" in state:
    state = state["model_state_dict"]

model.load_state_dict(state)
model.eval()

print("🔥 EfficientNet-B0 chargé !")

🔥 EfficientNet-B0 chargé !


C:\Users\ENNHILI YASSINE\AppData\Local\Temp\ipykernel_22604\3176927143.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(ckpt_path, map_location=device)

In [10]:
def evaluate_model(model, test_loader):
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for img_global, patches_local, labels in test_loader:

            img_global = img_global.to(device)
            labels = labels.to(device)

            # EfficientNet ne prend que l'image globale
            outputs = model(img_global)
            preds = outputs.argmax(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    f1_macro = f1_score(all_labels, all_preds, average="macro")
    precision_macro = precision_score(all_labels, all_preds, average="macro")
    recall_macro = recall_score(all_labels, all_preds, average="macro")

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
    }

In [11]:
metrics = evaluate_model(model, test_loader)

print("📊 Résultats EfficientNet-B0")
print(f"Accuracy   : {metrics['accuracy']*100:.2f}%")
print(f"F1-macro   : {metrics['f1_macro']:.4f}")
print(f"Precision  : {metrics['precision_macro']:.4f}")
print(f"Recall     : {metrics['recall_macro']:.4f}")

📊 Résultats EfficientNet-B0
Accuracy   : 91.07%
F1-macro   : 0.8959
Precision  : 0.9023
Recall     : 0.9015
